In [1]:
import numpy as np
import sys 


from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy_sp import *
from sympy.physics.mechanics import *
from sympy import *
from Data_generator_py import image_process
import sympy
import torch
import sys
import HLsearch as HL
import example_pendulum
import time

(1, 500, 51, 51)


In [2]:
save = False
environment = "server"
sample_size = 10
device = 'cuda:0'

In [3]:
sys.path.append(r'../../../HLsearch/')
#Saving Directory
params = {}
params['adding_noise'] = False
params['noise_type'] = 'image_noise'
params['noiselevel'] = 1e-1
params['changing_length'] = True
params['c'] = 1.4e-4
params['g'] = 9.81
params['l'] = 1
if environment == 'laptop':
    root_dir =R'C:\Users\87106\OneDrive\sindy\progress'
elif environment == 'desktop':
    root_dir = R'E:\OneDrive\sindy\progress'
elif environment == 'server':
    root_dir = R'/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression'
x,dx,ddx = image_process(sample_size,params)
X = []
Xdot = []
for i in range(len(x)):
    temp_list = [float(x[i]),float(dx[i])]
    X.append(temp_list)
    temp_list = [float(dx[i]),float(ddx[i])]
    Xdot.append(temp_list)
X = np.vstack(X)
print(X.shape)
Xdot = np.vstack(Xdot)

(4998, 2)


In [4]:
states_dim = 2
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

states are: (x0, x0_t)
states derivatives are:  (x0_t, x0_tt)


In [5]:
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

In [6]:
#build function expression for the library in str
expr= HL.buildFunctionExpressions(2,states_dim,states,use_sine=True)
#expr=['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
"a list of candidate function"
print(expr)
expr.pop(5)

#library function expression for the dissipation function in str
d_expr = [ 'x0_t','x0_t**2','x0_t**3']

['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


In [7]:
Zeta, Eta, Delta, Dissip = LagrangianLibraryTensor(X,Xdot,expr,d_expr,states,states_dot,device,scaling=True)
Eta = Eta.to(device)
Zeta = Zeta.to(device)
Delta = Delta.to(device)
Dissip = Dissip.to(device)

d_qdot [1, 2*x0_t, 3*x0_t**2]
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0') cuda:0
tensor([ 0.6201,  0.4699,  0.3027,  ..., -1.5455, -1.7719, -1.9971],
       device='cuda:0', dtype=torch.float64) cuda:0
tensor([0.2884, 0.1656, 0.0687,  ..., 1.7914, 2.3547, 2.9914], device='cuda:0',
       dtype=torch.float64) cuda:0


In [8]:
#coefficients for the Lagrangian
mask = torch.ones(len(expr),device=device)
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()
# coefficients for the dissipation function
d_mask = torch.ones(len(d_expr),device=device)
xi_d = torch.ones(len(d_expr),device=device).data.uniform_(-10,10)
prevxi_d = xi_d.clone().detach()

In [9]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [10]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

In [11]:
def proxL1norm(w_hat, alpha):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    return w

In [12]:
def training_loop(coef,d_coef, prevcoef, predcoef, Zeta, Eta, Delta,Dissip, xdot, bs, lr, lam):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]
    momentum = True
    #if(torch.is_tensor(xdot)==False):
        #xdot = torch.from_numpy(xdot).to(device).float()
    v = coef.clone().detach().to(device).requires_grad_(True)
    v_d = d_coef.clone().detach().to(device).requires_grad_(True)
    prev = prevcoef.clone().detach().to(device).requires_grad_(True)
    prev_d = prevdcoef.clone().detach().to(device).requires_grad_(True)
    for i in range(tl//bs):
        #computing acceleration with momentum
        #if (momentum == True):
        vhat = (v + ((i - 1) / (i + 2)) * (v - prev)).clone().detach().requires_grad_(True)
        vdhat = (v_d + ((i - 1) / (i + 2)) * (v_d - prev_d)).clone().detach().requires_grad_(True)
        #vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
        prev = v
        prev_d = v_d
        #Computing loss
        zeta = Zeta[:,i*bs:(i+1)*bs]
        eta = Eta[:,i*bs:(i+1)*bs]
        delta = Delta[:,i*bs:(i+1)*bs]
        dissip = Dissip[:,i*bs:(i+1)*bs]
        x_t = xdot[i*bs:(i+1)*bs,:]
        #forward
        q_tt_pred = lagrangianforward(vhat,vdhat,zeta,eta,delta,dissip,x_t,device)
        q_tt_true = xdot[i*bs:(i+1)*bs,n//2:].T
        q_tt_true = torch.from_numpy(q_tt_true).to(device).float()
        lossval = loss(q_tt_pred, q_tt_true)
        lossval.requires_grad_(True)
        #Backpropagation
        lossval.backward()
        with torch.no_grad():
            v = vhat - lr * vhat.grad
            v = proxL1norm(v, lr * lam)
            v_d = vdhat - lr * vdhat.grad
            v_d = proxL1norm(v_d, lr * lam)
            # Manually zero the gradients after updating weights
            vhat.grad = None
            vdhat.grad = None
        loss_list.append(lossval.item())
    return v, v_d, prev, prev_d, torch.tensor(loss_list).mean().item()

In [13]:
Epoch = 100
i = 1
lr = 1e-3
lam = 0.1
temp = 1000
while(i<=Epoch):
    xi_L , xi_d, prevxi_L, prevxi_d, lossitem= training_loop(xi_L,xi_d,prevxi_L,prevxi_d,Zeta,Eta,Delta,Xdot,128,lr=lr,lam=lam)
    if i %20 == 0:
        print("\n")
        print("Stage 1")
        print("Epoch "+str(i) + "/" + str(Epoch))
        print("Learning rate : ", lr)
        print("Average loss : " , lossitem)
    if(temp <=5e-3):
        break
    if(temp <=1e-1):
        lr = 1e-5
    temp = lossitem
    i+=1



Stage 1
Epoch 20/100
Learning rate :  0.001
Average loss :  29.1403751373291


Stage 1
Epoch 40/100
Learning rate :  0.001
Average loss :  4.471088409423828


Stage 1
Epoch 60/100
Learning rate :  0.001
Average loss :  2.585818290710449


Stage 1
Epoch 80/100
Learning rate :  0.001
Average loss :  2.0875608921051025


Stage 1
Epoch 100/100
Learning rate :  0.001
Average loss :  1.724971890449524


In [14]:
threshold = 1e-2
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()
surv_index_d = ((torch.abs(xi_d) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
d_expr = np.array(d_expr)[surv_index_d].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
xi_d =xi_d[surv_index_d].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()
prevxi_d = xi_d.clone().detach()
mask = torch.ones(len(expr),device=device)

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
xi_dcpu = np.around(xi_d.detach().cpu().numpy(),decimals=2)
L = HL.generateExpression(xi_Lcpu,expr)
D = HL.generateExpression(xi_dcpu,d_expr)
print(len(xi_L))
print(L)
print(D)

11
1.66*x0 + -2.72*sin(x0) + 12.14*cos(x0) + 1.84*x0**2 + 0.68*x0_t**2 + -4.82*x0*sin(x0) + 0.04*x0_t*sin(x0) + -5.96*sin(x0)**2 + 7.85*x0*cos(x0) + -5.8*x0_t*cos(x0) + -4.21*sin(x0)*cos(x0) + 


In [15]:
for stage in range(250):

    #Redefine computation after thresholding
    Zeta, Eta, Delta, Dissip = LagrangianLibraryTensor(X,Xdot,expr,d_expr,states,states_dot,device,scaling=True)
    Eta = Eta.to(device)
    Zeta = Zeta.to(device)
    Delta = Delta.to(device)
    Dissip = Dissip.to(device)

    #Training
    Epoch = 200
    i = 1
    lr = 1e-5
    if(stage==1):
        lam = 0
    else:
        lam = 0.1
    temp = 1000
    if stage >= 170:
        lr = 1e-6
    while(i<=Epoch):   
        xi_L , xi_d, prevxi_L, prevxi_d, lossitem= training_loop(xi_L,xi_d,prevxi_L,prevxi_d,Zeta,Eta,Delta,Xdot,128,lr=lr,lam=lam)
        if i %50 == 0:
            print("\n")
            print("Stage ",stage)
            print("Epoch "+str(i) + "/" + str(Epoch))
            print("Learning rate : ", lr)
            print("Average loss : " , lossitem)
        temp = lossitem
        if(temp <=1e-6):
            break
        i+=1
    
    ## Thresholding small indices ##
    threshold = 2e-1
    surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    expr = np.array(expr)[surv_index].tolist()
    surv_index_d = ((torch.abs(xi_d) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    d_expr = np.array(d_expr)[surv_index_d].tolist()

    xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
    xi_d =xi_d[surv_index_d].clone().detach().requires_grad_(True)
    prevxi_L = xi_L.clone().detach()
    prevxi_d = xi_d.clone().detach()
    mask = torch.ones(len(expr),device=device)

    ## obtaining analytical model
    xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
    xi_dcpu = np.around(xi_d.detach().cpu().numpy(),decimals=2)
    L = HL.generateExpression(xi_Lcpu,expr)
    D = HL.generateExpression(xi_dcpu,d_expr)
    print("expression length:\t",len(xi_L))
    print("Result stage " + str(stage+2) + ":" , L)
    print("Dissipation function": D)



Stage  0
Epoch 50/200
Learning rate :  1e-05
Average loss :  3.0039851665496826


Stage  0
Epoch 100/200
Learning rate :  1e-05
Average loss :  2.8566794395446777


Stage  0
Epoch 150/200
Learning rate :  1e-05
Average loss :  2.7247416973114014


Stage  0
Epoch 200/200
Learning rate :  1e-05
Average loss :  2.606511354446411
expression length:	 10
Result stage 2: 1.527*x0 + -2.826*sin(x0) + 12.162*cos(x0) + 1.577*x0**2 + 1.229*x0_t**2 + -4.818*x0*sin(x0) + -5.844*sin(x0)**2 + 7.572*x0*cos(x0) + -5.76*x0_t*cos(x0) + -4.371*sin(x0)*cos(x0) + 


Stage  1
Epoch 50/200
Learning rate :  1e-05
Average loss :  2.50380539894104


Stage  1
Epoch 100/200
Learning rate :  1e-05
Average loss :  2.410598039627075


Stage  1
Epoch 150/200
Learning rate :  1e-05
Average loss :  2.3259778022766113


Stage  1
Epoch 200/200
Learning rate :  1e-05
Average loss :  2.2487308979034424
expression length:	 10
Result stage 3: 1.483*x0 + -2.922*sin(x0) + 12.233*cos(x0) + 1.343*x0**2 + 1.255*x0_t**2 + -4.868*x